# Transformer  (distilbert-base-uncased)

In [ ]:
import os
import glob

import numpy as np
import pandas as pd

#import cudf
#import cupy as cp
# import nvtabular as nvt

# from google.cloud import bigquery

import pandas as pd
# import dask.dataframe as dd
# from dask.distributed import Client
# from dask.diagnostics import ProgressBar
# client = Client(n_workers=4, threads_per_worker=4, memory_limit="1GB")
# client

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="../key/key.json"

## Transfromer

In [ ]:
import texthero as hero
from texthero import preprocessing
from sentence_transformers import SentenceTransformer, util
import nmslib 
import time
import datetime

In [ ]:
distilbert = SentenceTransformer('distilbert-base-uncased')


In [ ]:
df = pd.read_pickle('../data/data_10000.pickel')
# Add one "0" to allign the customer_id to it's appearance in the submission csv: 
df['article_id'] = df['article_id'].apply(lambda x: "0"+x)
df.head(1)

In [ ]:
embeddings = distilbert.encode(df['description'], convert_to_tensor=True)


In [ ]:
distilbert_index = nmslib.init(method='hnsw', space='cosinesimil')
distilbert_index.addDataPointBatch(embeddings)
distilbert_index.createIndex({'post': 2}, print_progress=True)

# Creating a Search Function

In [ ]:
def search_cloth(dataframe, userQuery):
    
    if dataframe is not None and userQuery is not None:
        df = dataframe.copy()
        query = distilbert.encode([userQuery], convert_to_tensor=True)
        ids, distances = distilbert_index.knnQuery(query, k=12)

    matches = [] 
   
    for i, j in zip(ids, distances):          
            matches.append({'article_id':df.article_id.values[i]
                        ,"article_count":df.article_count.values[i]
                        ,"price":df.Avg_price.values[i]
                        , 'description':df.description.values[i]
                        , 'detail_desc' : df.detail_desc.values[i]
                        , 'distance': j
                       })     
    return pd.DataFrame(matches)

In [ ]:
search_cloth(df,"0732413001")

# Assign recommendations to customer_ids

In [ ]:
# Load transaction dataset:
df_trans = pd.read_csv('../data/transactions_train.csv', dtype={'article_id':'string'})

In [ ]:
# Train test split (transactions without the last 4 weeks):
df_trans_train = df_trans.query('t_dat < "2020-08-26"').copy()
# Drop not necessary columns:
df_trans_red = df_trans_train.drop(columns=['t_dat', 'price', 'sales_channel_id']).copy()
# Generate wardrobe:
df_wardrobe = df_trans_red.groupby('customer_id')['article_id'].aggregate(lambda x: list(x)).reset_index()
# Reverse wardrobe article_ids to focus on the last bought items:
df_wardrobe['wardrobe_reverse'] = df_wardrobe['article_id'].apply(lambda x: list(reversed(x)))
# Cut wardrobe_reverse to max 3 article_ids:
df_wardrobe['wardrobe_max_3'] = df_wardrobe['wardrobe_reverse'].apply(lambda x: x[:3] if len(x) > 3 else x)
# Generate empty column for recommendation
df_wardrobe['reco'] = np.nan
# Drop not necessary columns
df_wardrobe.drop(columns=['article_id', 'wardrobe_reverse'], inplace=True)

df_wardrobe.head()

In [ ]:
# Clear memory
del [[df_trans,df_trans_red, df_trans_train]]
import gc
gc.collect()



In [ ]:
# Changed search function in order to speed up process in the recommendation-assignment:

def search_cloth_new(dataframe, userQuery):
    
    if dataframe is not None and userQuery is not None:
        df = dataframe.copy()
        query = distilbert.encode([userQuery], convert_to_tensor=True)
        ids, distances = distilbert_index.knnQuery(query, k=12)

    matches = [] 
   
    for i, j in zip(ids, distances):          
            matches.append({'article_id':df.article_id.values[i]
                        , 'distance': j
                       })     
    return pd.DataFrame(matches)

In [ ]:
search_cloth_new(df,"0706016001")

In [ ]:
# Clear reco-column, if recommendations should be re-build:
# df_wardrobe['reco'] = np.nan

In [ ]:
# Get a sample to speed up process:
df_sample = df_wardrobe.sample(n=200000, random_state=42).copy()

In [ ]:
# Generate recommendations for each customer_id in dataframe:

from tqdm import tqdm

# Iterate over wardrobes:
for i, element in enumerate(tqdm(df_sample.wardrobe_max_3)):
    no_to_reco = int(12/len(element))
    recos = []
    for j, item in enumerate(element):
        # Store articles to recommend based on distance on certian article_id:
        reco_art = search_cloth_new(df, item)
        # Store recommendations per article_id in list:
        reco_on_wr = reco_art.nsmallest(no_to_reco, 'distance').article_id.to_list()
        # Append the recommendations to one list:
        recos.append(reco_on_wr)
    # Flatten the recos-"list of lists":
    recos = [item for sublist in recos for item in sublist]
    # Store recommendations in reco-column:
    df_sample['reco'].iloc[i] = recos


In [ ]:
df_sample.head()

In [ ]:
df_sample.to_csv('../data/20220510_Transformer_Recos_2000000_wardrobesize_1-3.csv', index = False)

# Generate submission csv

In [ ]:
# Create reco-column ("prediction") which stores recos in one string:
df_sample['prediction'] = df_sample.reco.apply(lambda x: ' '.join(x))
df_sample.head()

In [ ]:
# Generate submission train file (only customer_ids which exist in train dataset):
df_submission_train = df_sample.drop(columns=['wardrobe_max_3', 'reco'])
df_submission_train.head()

In [ ]:
# Load sample_submission to get all customer_ids and drop sample predictions:
df_sample_sub = pd.read_csv('../data/baseline_sample_submission.csv')
# Drop all rows where customer_ids have no recommendation:
df_sample_sub.drop(columns=['prediction'], inplace=True)
df_sample_sub.shape

In [ ]:
# Generate full submission file with all customer_ids:
df_submission_test = pd.merge(df_sample_sub, df_submission_train, how= 'left', on="customer_id")
# df_submission_test_100000 = df_submission_test.dropna(axis=0).copy()
df_submission_test_200000 = df_submission_test.copy()
df_submission_test_200000.head()

In [ ]:
# Transformer without baseline and 200000 customer_ids: Store submission file as csv:
df_submission_test_200000.to_csv('../data/Transformer-wo-Baseline-200000_20220510_submission_wardrobesize_1-3.csv', index = False)

# Generate submission csv with baseline fillup

In [ ]:
# Fill up with baseline-model recommendations:

df_submission_test_200000_baseline = df_submission_test.copy()
baseline = "0706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001"
df_submission_test_200000_baseline['prediction'].fillna(value=baseline, inplace=True)
df_submission_test_200000_baseline

In [ ]:
# Transformer 50000 customer_ids and others filled up with baseline recommendations: Store submission file as csv:
df_submission_test_200000_baseline.to_csv('../data/Transformer-Baseline-200000_20220510_submission_wardrobesize_1-3.csv', index = False)

# X.0 BACKUP

Old code backup

In [ ]:
# # Iterate over wardrobes:
# for i in tqdm(range(len(df_wardrobe))):
# # for i in tqdm(range(10000)):
#     # Calculate and store how much articles should be recommended for each ingoing article_id:
#     no_to_reco = int(12/len(df_wardrobe.wardrobe_max_3.iloc[i]))
#     recos = []
#     for j in range(len(df_wardrobe.wardrobe_max_3.iloc[i])):
#         # Store articles to recommend based on distance on certian article_id:
#         reco_art = search_cloth(df, df_wardrobe.wardrobe_max_3.iloc[i][j])
#         # Store recommendations per article_id in list:
#         reco_on_wr = reco_art.nsmallest(no_to_reco, 'distance').article_id.to_list()
#         # Append the recommendations to one list:
#         recos.append(reco_on_wr)
#     # Flatten the list of lists:
#     recos = [item for sublist in recos for item in sublist]
#     # Store recommendations in reco-column in wardrobe:
#     df_wardrobe['reco'].iloc[i] = recos

In [ ]:
# # OLD: only to test code

# # for i in tqdm(range(len(df_wardrobe))):
# for i in tqdm(range(5)):
#     recos = []
#     for j in range(len(df_wardrobe.wardrobe_max_3.iloc[i])):
#         # search_art = df_wardrobe.wardrobe_max_3.iloc[i][j]
#         reco_art = search_cloth(df, df_wardrobe.wardrobe_max_3.iloc[i][j])
#         if len(df_wardrobe.wardrobe_max_3.iloc[i]) == 3:
#             reco_on_3 = reco_art.nsmallest(4, 'distance').article_id.to_list()
#             recos.append(reco_on_3)
#             print(reco_on_3)
#         if len(df_wardrobe.wardrobe_max_3.iloc[i]) == 2:
#             reco_on_2 = reco_art.nsmallest(6, 'distance').article_id.to_list()
#             recos.append(reco_on_2)
#             print(reco_on_2)
#         if len(df_wardrobe.wardrobe_max_3.iloc[i]) == 1:
#             reco_on_1 = reco_art.nsmallest(12, 'distance').article_id.to_list()
#             recos.append(reco_on_1)
#             print(reco_on_1)
#     recos = [item for sublist in recos for item in sublist]
#     df_wardrobe['reco'].iloc[i] = recos
#     print(f'recolist is: {recos}')